In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics

import os, sys
from time import time

from phm08ds.models import experiment

## Load Dataset

In [2]:
folderpath = '../../../data/interim/'
data_op_03 = pd.read_csv(folderpath + 'data_op_03.csv')
data_op_03.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Sensor_15,Operational_condition,Health_state
1,1,2,0.0015,0.0003,100.0,1584.55,1403.96,14.62,2388.01,521.81,2388.15,0.03,3,1
7,1,8,0.0011,0.0000,100.0,1589.55,1407.59,14.62,2388.10,521.74,2388.03,0.03,3,1
8,1,9,0.0011,0.0020,100.0,1586.18,1399.01,14.62,2388.10,521.72,2388.08,0.03,3,1
16,1,17,0.0003,0.0000,100.0,1584.57,1410.11,14.62,2388.06,522.17,2388.10,0.03,3,1
19,1,20,0.0004,0.0017,100.0,1581.33,1399.44,14.62,2388.09,521.62,2388.06,0.03,3,1


## Data preprocessing

### Get rid of informations there are not sensor readings

Wang (2008) reports Sensor 15 has importat information. However, there are no relevant informations of this sensor. The data seems to be corrupted like this:

Let's remove it from our database creating an object transformer.

In [3]:
from phm08ds.features.feature_selection import RemoveSensor

tf_remove_sensor_15 = RemoveSensor(sensors=[15])
data_op_03 = tf_remove_sensor_15.fit_transform(data_op_03)
data_op_03.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Operational_condition,Health_state
1,1,2,0.0015,0.0003,100.0,1584.55,1403.96,14.62,2388.01,521.81,2388.15,3,1
7,1,8,0.0011,0.0000,100.0,1589.55,1407.59,14.62,2388.10,521.74,2388.03,3,1
8,1,9,0.0011,0.0020,100.0,1586.18,1399.01,14.62,2388.10,521.72,2388.08,3,1
16,1,17,0.0003,0.0000,100.0,1584.57,1410.11,14.62,2388.06,522.17,2388.10,3,1
19,1,20,0.0004,0.0017,100.0,1581.33,1399.44,14.62,2388.09,521.62,2388.06,3,1


Before feeding to the classifier, let's remove unwanted information, such as unit, time_step and operational settings.

In [4]:
from phm08ds.features.feature_selection import RemoveInfo

tf_remove_info = RemoveInfo()

data_with_features = tf_remove_info.fit_transform(data_op_03)
data_with_features.head()

,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Health_state
1,1584.55,1403.96,14.62,2388.01,521.81,2388.15,1
7,1589.55,1407.59,14.62,2388.10,521.74,2388.03,1
8,1586.18,1399.01,14.62,2388.10,521.72,2388.08,1
16,1584.57,1410.11,14.62,2388.06,522.17,2388.10,1
19,1581.33,1399.44,14.62,2388.09,521.62,2388.06,1


We need to normalize our data. Let's use Z-score standardization.

In [5]:
from sklearn.preprocessing import StandardScaler

tf_std_scaller = preprocessing.StandardScaler()
data_with_features_std = tf_std_scaller.fit_transform(data_with_features.drop(labels='Health_state', axis=1))
data_with_features_std

array([[-1.040896  , -0.62635123, -1.        , -1.37994051,  0.63774241,
         0.65278365],
       [-0.19248051, -0.21074616, -1.        , -0.05996073,  0.5391007 ,
        -1.09204307],
       [-0.76431255, -1.19308541, -1.        , -0.05996073,  0.51091736,
        -0.36503194],
       ...,
       [ 0.79168145,  0.75556424, -1.        ,  1.55334789, -1.33509174,
         2.39761038],
       [ 1.7639656 ,  1.08530049, -1.        ,  2.4333344 , -2.16650042,
         2.97921928],
       [ 1.69609236,  1.87872835, -1.        ,  2.4333344 , -1.67329188,
         1.81600147]])

In [6]:
labels = np.array(data_with_features['Health_state'])
labels

array([1, 1, 1, ..., 4, 4, 4])

# Classification steps

## Load Experiment model

In [7]:
from phm08ds.models import experiment

## Define classifiers and its specifications

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [9]:
# SVM
svm_linear_clf = SVC(kernel='linear')
svm_rbf_clf = SVC(kernel='rbf')
svm_poly_clf = SVC(kernel='poly')
svm_sigmoid_clf = SVC(kernel='sigmoid')

## Put all clf in a dictionary:

In [10]:
classifiers = {'SVM-Linear': svm_linear_clf, 'SVM-RBF': svm_rbf_clf, 'SVM-Poly': svm_poly_clf, 'SVM-Sigmoid': svm_sigmoid_clf}

Since we are using SVM and MLP we need to extract all power from those methods. Let's perform a Random Search to parameters optimizations.

### Hyperparameter tunning

In [11]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('remove_sensor_15', tf_remove_sensor_15),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [12]:
from sklearn.model_selection import RandomizedSearchCV

random_search = dict((k,[]) for k in classifiers.keys())

In [13]:
param_dist_dict = {
                   'SVM-Linear': {'C': [2**i for i in range(-5,15)]},
                   'SVM-RBF': {'gamma': [2**i for i in range(-15,3)], 'C': [2**i for i in range(-5,15)]},
                   'SVM-Poly': {'gamma': [2**i for i in range(-15,3)], 'C': [2**i for i in range(-5,15)]},
                   'SVM-Sigmoid': {'gamma': [2**i for i in range(-15,3)], 'C': [2**i for i in range(-5,15)]}
                  }

In [14]:
for clf in param_dist_dict.keys():
    start = time()
    random_search[clf] = RandomizedSearchCV(classifiers[clf], param_dist_dict[clf], cv=10, n_iter=5, verbose=5, n_jobs=1, scoring='accuracy')
    random_search[clf].fit(data_with_features_std, labels)
    
    experiment.save_models(random_search, name='clf_svm')
    experiment.save_pipeline(data_preprocessing)
    
    print('Elapsed time:')
    print(time() - start)

Fitting 10 folds for each of 5 candidates, totalling 50 fits
[CV] C=0.03125 .......................................................
[CV] .............. C=0.03125, score=0.6097560975609756, total=   0.5s
[CV] C=0.03125 .......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


[CV] .............. C=0.03125, score=0.6226685796269728, total=   0.5s
[CV] C=0.03125 .......................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.6s remaining:    0.0s


[CV] .............. C=0.03125, score=0.6183644189383071, total=   0.5s
[CV] C=0.03125 .......................................................


...

[CV]  gamma=0.0078125, C=1024, score=0.46551724137931033, total=   1.2s
[CV] gamma=0.0078125, C=1024 .........................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    7.5s remaining:    0.0s


[CV]  gamma=0.0078125, C=1024, score=0.4848920863309353, total=   1.2s
[CV] gamma=0.0078125, C=1024 .........................................
[CV]  gamma=0.0078125, C=1024, score=0.5007194244604316, total=   1.2s
[CV] gamma=0.0078125, C=1024 .........................................
[CV]  gamma=0.0078125, C=1024, score=0.46906474820143884, total=   1.3s
[CV] gamma=0.0078125, C=1024 .........................................
[CV] . gamma=0.0078125, C=1024, score=0.515850144092219, total=   1.2s
[CV] gamma=0.0078125, C=1024 .........................................
[CV]  gamma=0.0078125, C=1024, score=0.5144092219020173, total=   1.2s
[CV] gamma=0.0078125, C=1024 .........................................
[CV] . gamma=0.0078125, C=1024, score=0.484149855907781, total=   1.2s
[CV] gamma=0.0009765625, C=16 ........................................
[CV]  gamma=0.0009765625, C=16, score=0.6126255380200861, total=   1.0s
[CV] gamma=0.0009765625, C=16 ........................................
[CV]

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  2.1min finished


Elapsed time:
126.41310143470764


## Savel results, models and pipeline to a .pkl file 

In [15]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('remove_sensor_15', tf_remove_sensor_15),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [16]:
experiment.save_models(random_search, name='clf_svm')
experiment.save_pipeline(data_preprocessing)